In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import requests
import numpy as np
model = VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None)

/Users/shohei.numahata/.pyenv/versions/3.6.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/shohei.numahata/.pyenv/versions/3.6.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/shohei.numahata/.pyenv/versions/3.6.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/shohei.nu

In [2]:
model.save('vgg16_model.h5')

In [3]:
converter = tf.lite.TFLiteConverter.from_keras_model_file("vgg16_model.h5")
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

W1004 14:39:00.610852 4567692736 deprecation.py:506] From /Users/shohei.numahata/.pyenv/versions/3.6.6/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1004 14:39:00.611887 4567692736 deprecation.py:506] From /Users/shohei.numahata/.pyenv/versions/3.6.6/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1004 14:39:01.938213 4567692736 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.
W1004 14:39:02

553436732

In [4]:
# TFLiteモデルの読み込み
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
# メモリ確保。これはモデル読み込み直後に必須
interpreter.allocate_tensors()

# 学習モデルの入力層・出力層のプロパティをGet.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [5]:
print(input_details)
print(output_details)

[{'name': 'input_1', 'index': 50, 'shape': array([  1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
[{'name': 'predictions/Softmax', 'index': 53, 'shape': array([   1, 1000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]


In [6]:
#画像をダンロードするための関数
def download_img(url, file_name):
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        with open(file_name, 'wb') as f:
            f.write(r.content)

In [16]:
if __name__ == '__main__':
    #画像のダウンロード
    url = 'https://cdn.pixabay.com/photo/2016/03/05/19/02/hamburger-1238246_1280.jpg'
    file_name = 'hamburger.jpg'
    download_img(url, file_name)
    img = image.load_img(file_name, target_size=(224, 224))
    
    # 読み込んだPIL形式の画像をarrayに変換
    input_data = image.img_to_array(img)

    #サンプル数の次元を1つ増やし四次元テンソルに
    input_data = np.expand_dims(input_data, axis=0)
    
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    interpreter.invoke()
    
    #推論結果はoutput_dataのindexに保存されている
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print(output_data)

    #上位5を出力
    #推論
    results = decode_predictions(output_data, top=5)[0]
    for result in results:
        print(result)

[[1.68088992e-08 2.02147612e-06 3.31995693e-07 1.20214267e-07
  7.76728459e-09 2.12742952e-06 1.47145258e-07 6.89589852e-09
  6.76742005e-08 2.73908469e-08 8.73372130e-08 1.00550309e-07
  3.99138465e-08 6.02208843e-08 4.97785038e-07 3.62851549e-09
  8.28946956e-09 1.74592088e-07 1.54536239e-09 1.63362344e-07
  2.35937918e-08 1.04510212e-09 1.82480708e-09 4.53961269e-09
  1.29862610e-08 6.22849683e-08 2.51901469e-07 1.32698702e-07
  1.44252581e-07 1.54131212e-05 1.29485485e-07 1.36306448e-06
  7.76592174e-07 4.79206983e-06 1.29377554e-06 4.14420327e-07
  7.30119655e-06 2.15773440e-07 1.09015514e-06 1.37552085e-08
  1.45062176e-08 9.96695726e-10 1.82316517e-09 5.83416981e-10
  7.26344673e-09 6.75641232e-08 8.12485812e-09 7.60255574e-08
  5.22970611e-10 4.78408744e-08 1.82179051e-07 1.77328232e-08
  6.74804781e-08 8.63602523e-09 7.07552914e-08 9.29600930e-09
  5.11631768e-08 6.60897559e-10 1.84878157e-08 4.16557846e-08
  1.08262967e-07 3.73822440e-09 5.86230193e-08 2.83857613e-08
  2.7484

In [9]:
max_label = np.argmax(output_data)
print(max_label)

933
